<img align="right" alt="PaLM 2" src="https://i.imgur.com/KAEWX6i.jpg?raw=true" width="1092" height="614" />

## Pair Programming with the Google PaLM LLM AI Model Bison

## Get the API key

In [1]:
# Check the Python environment packages
#!conda list
#!pip list

# Install the Python packages
#!conda install python-dotenv
#!pip install python-dotenv

# Imports
import os
from dotenv import load_dotenv, find_dotenv
#from utils import get_api_key

# Get the API key
def get_api_key():
    
    _ = load_dotenv(find_dotenv()) # read local .env file
    return os.getenv('GOOGLE_API_KEY')

# Test printing the API key
if get_api_key():
    print("The API key is valid")

The API key is valid


## Connect to the Google PaLM LLM AI Model

In [2]:
# Install the Google PaLM relevant libraries
#!pip install -q google.generativeai

# Imports
import os
import google.generativeai as palm
from google.api_core import client_options as client_options_lib

# Connect to the Google PaLM LLM AI Model
palm.configure(
    api_key=get_api_key(),
    transport="rest",
    client_options=client_options_lib.ClientOptions(
        api_endpoint=os.getenv("GOOGLE_API_BASE"),
    )
)

for m in palm.list_models():
    print(f"name: {m.name}")
    print(f"description: {m.description}")
    print(f"generation methods:{m.supported_generation_methods}\n")

name: models/chat-bison-001
description: Chat-optimized generative language model.
generation methods:['generateMessage', 'countMessageTokens']

name: models/text-bison-001
description: Model targeted for text generation.
generation methods:['generateText', 'countTextTokens', 'createTunedTextModel']

name: models/embedding-gecko-001
description: Obtain a distributed representation of a text.
generation methods:['embedText']



## Filter models by their supported generation methods

In [3]:
# Filter the text generation models from the list
#models = [m for m in palm.list_models() 
#          if 'generateText' 
#          in m.supported_generation_methods]
#models

#model_bison = models[0]
#model_bison

# Filter the text generation models from the list
models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
model_bison = models[0]
#model_bison

## Helper function to generate text

In [4]:
# Imports
from google.api_core import retry

# define a retry decorator
@retry.Retry()

# Helper function to generate text
def generate_text(prompt,
                  model=model_bison,
                  temperature=0.0):
    return palm.generate_text(prompt=prompt,
                              model=model,
                              temperature=temperature)

## Prompt templates

In [122]:
**Prompts list**:
- Prompt using a simple prompt
- Prompt using a template
  - Pair Programming Scenarios
    - Improve existing code
    - Simplify code
    - Write test cases
    - Make code more efficient
    - Debug your code
  - Technical Debt
    - Ask an LLM to explain a complex code base (or your own code!)
    - Ask an LLM to document a complex code base (or your own code!)

## Prompt using a simple prompt

```python
prompt = "Show me how to iterate across a list in Python."
#prompt = "How updated are you as a model?"  # 2023-03-08
#prompt = "Can you connect you to the internet"  # yes
#prompt = "What is the URL of teh channel lescifi on YouTube?"  # Wrong answer!
#prompt = "How is the weather in NY?"
#prompt = "Where do you read the wether from?"  # weather.com
#prompt = "What day and time is today?"  # 2023-03-08 10:00:00

# Generate text
completion = generate_text(prompt)
print(completion.result)

# Review the prompt
#print(prompt)

```

## Prompt using a template

```python
prompt_template = """
{priming}

{question}

{decorator}

Your solution:
"""

priming_text = "You are an expert at writing clear, concise, Python code."
question = "create a doubly linked list"
#decorator = "Work through it step by step, and show your work. One step per line."
decorator = "Insert comments for each line of code."

prompt = prompt_template.format(priming=priming_text,
                                question=question,
                                decorator=decorator)

# Generate text
completion = generate_text(prompt)
print(completion.result)

# Review the prompt
#print(prompt)

```

## Pair Programming Scenarios

## Improve existing code

```python
prompt_template = """
I don't think this code is the best way to do it in Python, can you help me?

{question}

Please explain, in detail, what you did to improve it.
"""

question = """
def func_x(array)
  for i in range(len(array)):
    print(array[i])
"""

completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.result)

# Review the prompt
#print(prompt)

```

## Simplify code

```python
# option 1
prompt_template = """
Can you please simplify this code for a linked list in Python?

{question}

Explain in detail what you did to modify it, and why.
"""

# option 2
prompt_template = """
Can you please simplify this code for a linked list in Python? \n
You are an expert in Pythonic code.

{question}

Please comment each line in detail, \n
and explain in detail what you did to modify it, and why.
"""

question = """
class Node:
  def __init__(self, dataval=None):
    self.dataval = dataval
    self.nextval = None

class SLinkedList:
  def __init__(self):
    self.headval = None

list1 = SLinkedList()
list1.headval = Node("Mon")
e2 = Node("Tue")
e3 = Node("Wed")
list1.headval.nextval = e2
e2.nextval = e3
"""

completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.result)

# Review the prompt
#print(prompt)

```

## Write test cases

```python
prompt_template = """
Can you please create test cases in code for this Python code?

{question}

Explain in detail what these test cases are designed to achieve.
"""

# Note that the code I'm using here was output in the previous
# section. Your output code may be different.
question = """
class Node:
  def __init__(self, dataval=None):
    self.dataval = dataval
    self.nextval = None

class SLinkedList:
  def __init__(self):
    self.head = None

def create_linked_list(data):
  head = Node(data[0])
  for i in range(1, len(data)):
    node = Node(data[i])
    node.nextval = head
    head = node
  return head

list1 = create_linked_list(["Mon", "Tue", "Wed"])
"""

completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.result)

# Review the prompt
#print(prompt)

```

## Make code more efficient

```python
prompt_template = """
Can you please make this code more efficient?

{question}

Explain in detail what you changed and why.
"""

question = """
# Returns index of x in arr if present, else -1
def binary_search(arr, low, high, x):
    # Check base case
    if high >= low:
        mid = (high + low) // 2
        if arr[mid] == x:
            return mid
        elif arr[mid] > x:
            return binary_search(arr, low, mid - 1, x)
        else:
            return binary_search(arr, mid + 1, high, x)
    else:
        return -1

# Test array
arr = [ 2, 3, 4, 10, 40 ]
x = 10

# Function call
result = binary_search(arr, 0, len(arr)-1, x)

if result != -1:
    print("Element is present at index", str(result))
else:
    print("Element is not present in array")

"""

completion = generate_text(
    prompt = prompt_template.format(question=question)
)
print(completion.result)

# Review the prompt
#print(prompt)

```

## Debug your code

```python
prompt_template = """
Can you please help me to debug this code?

{question}

Explain in detail what you found and why it was a bug.
"""

# I deliberately introduced a bug into this code! Let's see if the LLM can find it.
# Note -- the model can't see this comment -- but the bug is in the
# print function. There's a circumstance where nodes can be null, and trying
# to print them would give a null error.
question = """
class Node:
   def __init__(self, data):
      self.data = data
      self.next = None
      self.prev = None

class doubly_linked_list:
   def __init__(self):
      self.head = None

# Adding data elements
   def push(self, NewVal):
      NewNode = Node(NewVal)
      NewNode.next = self.head
      if self.head is not None:
         self.head.prev = NewNode
      self.head = NewNode

# Print the Doubly Linked list in order
   def listprint(self, node):
       print(node.data),
       last = node
       node = node.next

dllist = doubly_linked_list()
dllist.push(12)
dllist.push(8)
dllist.push(62)
dllist.listprint(dllist.head)

"""

# Temperature = 0.0 results in more deterministic output
# Since a temperature > 0 encourages more randomness in the LLM output, you may
# want to run this code a couple times to see what it outputs.
completion = generate_text(
    prompt = prompt_template.format(question=question),
    temperature = 0.7
)
print(completion.result)

# Review the prompt
#print(prompt)

```

## Technical Debt

## Ask an LLM to explain a complex code base (or your own code!)

```python
#@title Complex Code Block
# Note: Taken from https://github.com/lmoroney/odmlbook/blob/63c0825094b2f44efc5c4d3226425a51990e73d6/BookSource/Chapter08/ios/cats_vs_dogs/CatVsDogClassifierSample/ModelDataHandler/ModelDataHandler.swift
CODE_BLOCK = """
// Copyright 2019 The TensorFlow Authors. All Rights Reserved.
//
// Licensed under the Apache License, Version 2.0 (the "License");
// you may not use this file except in compliance with the License.
// You may obtain a copy of the License at
//
//    http://www.apache.org/licenses/LICENSE-2.0
//
// Unless required by applicable law or agreed to in writing, software
// distributed under the License is distributed on an "AS IS" BASIS,
// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
// See the License for the specific language governing permissions and
// limitations under the License.

import CoreImage
import TensorFlowLite
import UIKit


/// An inference from invoking the `Interpreter`.
struct Inference {
  let confidence: Float
  let label: String
}

/// Information about a model file or labels file.
typealias FileInfo = (name: String, extension: String)

/// Information about the MobileNet model.
enum MobileNet {
  static let modelInfo: FileInfo = (name: "converted_model", extension: "tflite")
}

# More large and complex code here ...

"""

prompt_template = """
Can you please explain how this code works?

{question}

Use a lot of detail and make it as clear as possible.
"""

# Keeping the prompt simple, just to show the LLM the Complex Code Block
completion = generate_text(
    prompt = prompt_template.format(question=CODE_BLOCK)
)
print(completion.result)

# Review the prompt
#print(prompt)

```

## Ask an LLM to document a complex code base (or your own code!)

```python
CODE_BLOCK = """ 
# replace this with your own code
def foo(a):
  b = a + 1
  return 2*b
"""

prompt_template = """
Please write technical documentation for this code and \n
make it easy for a non Python developer to understand:

{question}

Output the results in markdown
"""

completion = generate_text(
    prompt = prompt_template.format(question=CODE_BLOCK)
)
print(completion.result)

# Review the prompt
#print(prompt)

```


SyntaxError: invalid syntax (494728963.py, line 1)

## Create your prompt in the following cell

* **Hide the cell Prompt Templates before running all the cells**
* **Check and run the results after they are generated**
* **Remember to import packages as needed to make a resultant code work**
----------------------- Start your prompts here... ----------------------

In [21]:
prompt = "How updated are you as a model?"
# Generate text
completion = generate_text(prompt)
print(completion.result)

# Review the prompt
#print(prompt)

2023-03-08
